In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf

In [2]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=d)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
ffK(s) = (@sprintf "%.5g" sd(3,s)/1e3)
ffM(s) = (@sprintf "%.3g" sd(3,s)/1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffe8(s) = (@sprintf "%.4g" sd(3,s)*1e8)

ffe8 (generic function with 1 method)

In [3]:
@load "variances.jld2"
dic1 = copy(dic);

In [4]:
wted_chimera_indices = findall(entry -> contains(entry, "wted_chimera(10000000,1)"), dic1["testName"])
sachedeva_indices = findall(entry -> contains(entry, "star_join(complete_graph(700), 350)"), dic1["testName"])
wted_bndry_chimera_indices = findall(entry -> contains(entry, "wted_bndry_chimera(10000000,1)"), dic1["testName"])
checkered_grid_indices = findall(entry -> contains(entry, "checkered_grid_sddm(2.0e8, 32, 32, 32, 1.0e7)"), dic1["testName"])
wgrid_indices = findall(entry -> contains(entry, "wgrid_sddm(2.0e8, 0.001)"), dic1["testName"]);

In [5]:
problem_idx = [wted_chimera_indices, wted_bndry_chimera_indices, sachedeva_indices, checkered_grid_indices, wgrid_indices]
instanceNames = ["Weighted chimera", "Weighted SDDM chimera", "Sachdeva Star", "High contrast coefficient Poisson grid", "Anisotropic coef. Poisson grid, variable weight"];


In [6]:
function extractSlice(dic, slice)
    dicSlice = Dict()
    numepxr = length(dic["ne"])
    for (key, value) in dic
        if length(value) == numepxr
            dicSlice[key] = value[slice]
        else
            dicSlice[key] = value
        end
    end
    return dicSlice
end

extractSlice (generic function with 1 method)

In [7]:
dics = [extractSlice(dic1, idx) for idx in problem_idx];

In [8]:
instanceNnzs = [maximum(dic1["ne"][indices]) for indices in problem_idx]


5-element Vector{Int64}:
  50163366
  49746425
 171500701
 200006496
 200006496

In [9]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver, category)
    if haskey(d, string(solver, category)) && length(d[string(solver, category)]) == length(d["ne"])
        tvn = median(d[string(solver, category)] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver, category)
    if haskey(d, string(solver, category)) && length(d[string(solver, category)]) == length(d["ne"])
        tvn = quantile(d[string(solver, category)] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver, category)
    if haskey(d, string(solver, category)) && length(d[string(solver, category)]) == length(d["ne"])
        tvn = maximum(d[string(solver, category)] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

printMax (generic function with 1 method)

In [10]:
function printSolverCell(d, solver, category) 
    return string(printMedian(d, solver, category), "&", print75(d, solver, category), "&", printMax(d, solver, category))
end

function printOneLine(d, solvers)
    allSolveCells = [printSolverCell(d, solver, "_solve") for solver in solvers]
    allTotCells = [printSolverCell(d, solver, "_tot") for solver in solvers]
    return string(join(allSolveCells, "&"), "&", join(allTotCells, "&"), "\\\\")
end

function printAllLines(instanceNames, instanceNnzs, dics, solvers)
    for i in 1:length(dics)
        println(instanceNames[i], "&", ffM(instanceNnzs[i]) , "&", printOneLine(dics[i], solvers))
    end
end

printAllLines (generic function with 1 method)

In [11]:
solvers = ["ac", "ac-s2m2"]
printAllLines(instanceNames, instanceNnzs, dics, solvers)

Weighted chimera&50.2&3.78&3.87&3.95&2.59&2.6&2.68&4.65&4.74&4.82&4.09&4.11&4.18\\
Weighted SDDM chimera&49.7&3.3&3.38&3.48&2.44&2.5&2.58&4.45&4.53&4.64&4.26&4.33&4.4\\
Sachdeva Star&172&4.04&4.48\tnote{*}&4.55\tnote{**}&0.198&0.201&0.209&4.46&4.9\tnote{*}&4.97\tnote{**}&0.832&0.836&0.846\\
High contrast coefficient Poisson grid&200&1.71&1.73&1.77&2.14&2.15&2.25&2.6&2.62&2.66&3.34&3.35&3.45\\
Anisotropic coef. Poisson grid, variable weight&200&1.12&1.14&1.18&1.15&1.16&1.2&1.72&1.75&1.82&2.33&2.34&2.38\\


In [12]:
ratios = []
errors = []
for indices in problem_idx
    ratio = dic1["rchol_tot"][indices] ./ dic1["ne"][indices]
    push!(ratios, ratio)
    error = dic1["rchol_err"][indices]
    push!(errors, error)
end


In [13]:
for (name, ratio, error) in zip(instanceNames, ratios, errors)
    mean_val = mean(ratio)
    quantile_val = quantile(ratio, 0.75)
    max_val = maximum(ratio)
    
    @printf("%-20s - Ratio - Mean: %10s | 0.75 Quantile: %10s | Max: %10s\n", name, ffmicro(mean_val), ffmicro(quantile_val), ffmicro(max_val))
    
    mean_error = mean(error)
    quantile_error = quantile(error, 0.75)
    max_error = maximum(error)
    
    @printf("\u001b[31m%-20s - Error - Mean: %10s | 0.75 Quantile: %10s | Max: %10s\n\u001b[0m", name, ffe8(mean_error), ffe8(quantile_error), ffe8(max_error))
end

Weighted chimera     - Ratio - Mean:       2.92 | 0.75 Quantile:       2.93 | Max:       3.04
Weighted chimera     - Error - Mean:      1e+08 | 0.75 Quantile:      1e+08 | Max:      1e+08
Weighted SDDM chimera - Ratio - Mean:       6.27 | 0.75 Quantile:       6.35 | Max:       6.46
Weighted SDDM chimera - Error - Mean:      0.965 | 0.75 Quantile:      0.979 | Max:       0.99
Sachdeva Star        - Ratio - Mean:      0.255 | 0.75 Quantile:      0.256 | Max:      0.258
Sachdeva Star        - Error - Mean:      1e+08 | 0.75 Quantile:      1e+08 | Max:      1e+08
High contrast coefficient Poisson grid - Ratio - Mean:       5.76 | 0.75 Quantile:       5.93 | Max:       6.04
High contrast coefficient Poisson grid - Error - Mean:      0.963 | 0.75 Quantile:      0.996 | Max:      0.998
Anisotropic coef. Poisson grid, variable weight - Ratio - Mean:        2.3 | 0.75 Quantile:       2.32 | Max:        2.4
Anisotropic coef. Poisson grid, variable weight - Error - Mean:       0.89 | 0.75 Quantil